In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [4]:
edges = cg.read_chunk_edges([cg.get_chunk_id(layer=2,x=50,y=100,z=0)])
edges = edges["between"].get_pairs()
edges

array([[75582903194484819, 75582834475009044],
       [75582903194484867, 75582834475009018],
       [75582903194484873, 75582834475009193],
       [75582903194484912, 75512534450307165],
       [75582903194484963, 75582834475009168],
       [75582903194485200, 75512534450307435],
       [75582903194485330, 75512534450307707],
       [75582903194485481, 75512534450307707],
       [75582903194485506, 75582971913961513],
       [75582903194485507, 75582971913961542],
       [75582903194485544, 75582971913961501],
       [75582903194485666, 75582971913961503],
       [75582903194485697, 75582971913961503],
       [75582903194485720, 75512534450307957],
       [75582903194485720, 75582971913961693],
       [75582903194485753, 75582971913961503]], dtype=uint64)

In [2]:
from pychunkedgraph.graph.edits_v2 import add_edge_v2
add_edge_v2(cg, edge=[75582903194484819, 75582903194485753])

bounding_chunk_ids [147640497232412672 147640497299521536 147640565951889408
 147640566018998272 147710865976590336 147710866043699200
 147710934696067072 147710934763175936]
node_id, children 219704963218014209 [147640497232412720 147640497232412673 147710934696067076
 147640497232412683 147640565951889409]

children_chunk_ids [147640497232412672 147640497232412672 147710934696067072
 147640497232412672 147640565951889408]
bounding_chunk_ids [219564225729658880 219564226266529792 219564500607565824
 219564501144436736 219704963218014208 219704963754885120
 219705238095921152 219705238632792064]
node_id, children 291635563662934017 [219704963218014209 219564225729658884 219704963218014230]

children_chunk_ids [219704963218014208 219564225729658880 219704963218014208]
bounding_chunk_ids [147499759744057344 147499759811166208 147499759878275072
 147499759945383936 147499828463534080 147499828530642944
 147499828597751808 147499828664860672 147499897183010816
 147499897250119680 147499897

bounding_chunk_ids [363718446468300800 363718480828039168 363722844514811904
 363722878874550272 364281396421722112 364281430781460480
 364285794468233216 364285828827971584]
node_id, children 435828817064361985 [363718446468300801]

children_chunk_ids [363718446468300800]
bounding_chunk_ids [291634464151306240 291634468446273536 291634472741240832
 291634477036208128 291635563662934016 291635567957901312
 291635572252868608 291635576547835904 291636663174561792
 291636667469529088 291636671764496384 291636676059463680
 291637762686189568 291637766981156864 291637771276124160
 291637775571091456 291915939128016896 291915943422984192
 291915947717951488 291915952012918784 291917038639644672
 291917051524546560 291918138151272448 291918151036174336
 291919237662900224 291919241957867520 291919246252834816
 291919250547802112 292197414104727552 292197418399694848
 292197422694662144 292197426989629440 292198513616355328
 292198526501257216 292199613127983104 292199626012884992
 2922007126

bounding_chunk_ids [434702917157519360 434703192035426304 434720509343563776
 434720784221470720 435828817064361984 435829091942268928
 435846409250406400 435846684128313344]
node_id, children 506866064311713793 [435828817064361985 435828817064694903 434702917157630643
 434702917157524103]

children_chunk_ids [435828817064361984 435828817064361984 434702917157519360
 434702917157519360]
bounding_chunk_ids [362592546561458176 362592580921196544 362592615280934912
 362592649640673280 362596944607969280 362596978967707648
 362597013327446016 362597047687184384 362601342654480384
 362601377014218752 362601411373957120 362601445733695488
 362605740700991488 362605775060729856 362605809420468224
 362605843780206592 363155496514879488 363155530874617856
 363155565234356224 363155599594094592 363159894561390592
 363159997640605696 363164292607901696 363164395687116800
 363168690654412800 363168725014151168 363168759373889536
 363168793733627904 363718446468300800 363718480828039168
 3637185151

node_id, children 506866064311713793 [219704963218014209 219564225729658884 219704963218014230
 219563675973844993 219704413462200321 219563950851751937
 219704688340107265 147640497232412674 219564225729658884
 219704963218014230]

children_chunk_ids [219704963218014208 219564225729658880 219704963218014208
 219563675973844992 219704413462200320 219563950851751936
 219704688340107264 147640497232412672 219564225729658880
 219704963218014208]
bounding_chunk_ids [146373584959307776 146373585026416640 146373585093525504 ...
 148557147876098048 148557147943206912 148557148010315776]
node_id, children 506866064311713793 [147640497232412720 147640497232412673 147710934696067076
 147640497232412683 147640565951889409 147499484866150401
 147499553585627137 147569853610328065 147569922329804801
 147499622305103873 147499691024580609 147569991049281537
 147570059768758273 147499759744057347 147570128488235009
 147570197207711746 147640222354505729 147640291073982465
 147640359793459201 14764042

node_id, children 576742227280134145 [291635563662934017 291634464151307295 291635563662936192
 291635563662934656 290507464732835842 290507464732835845
 290788939709546501 219421564095954945 291069315174629378
 291351889662967838 291069315174629380 291350790151340079
 291070414686257154 291631165616422943 291632265128050898
 291633364639679012 291634464151307295 291635563662936192
 291635563662934656]

children_chunk_ids [291635563662934016 291634464151306240 291635563662934016
 291635563662934016 290507464732835840 290507464732835840
 290788939709546496 219421564095954944 291069315174629376
 291351889662967808 291069315174629376 291350790151340032
 291070414686257152 291631165616422912 291632265128050688
 291633364639678464 291634464151306240 291635563662934016
 291635563662934016]
bounding_chunk_ids [216181578206806016 216181578743676928 216181579280547840 ...
 220552966929645568 220552967466516480 220552968003387392]
node_id, children 576742227280134145 [219704963218014209 21956422

[864691128455135233]